In [15]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import torch.optim as optim
import sklearn
from sklearn import metrics as metrics
import pandas as pd

# Task 1
#### Completing `bust_probability` function

In [16]:
def data_loader(path, table_idx, player_or_dealer):
    #utility for loading train.csv, example use in the notebook
    data = pd.read_csv(path, header=[0,1,2])
    spy = data[(f'table_{table_idx}', player_or_dealer, 'spy')]
    card = data[(f'table_{table_idx}', player_or_dealer, 'card')]
    return np.array([spy, card]).T

In [20]:
table_index=3
dealer_data = data_loader("data/train.csv", table_index, "dealer")
print(dealer_data.tolist())
dealer_cards = dealer_data[:,1]

[[50.93, 3.0], [52.33, 4.0], [53.6, 6.0], [54.69, 7.0], [55.92, 8.0], [57.02, 9.0], [58.31, 10.0], [59.69, 2.0], [60.48, 2.0], [61.23, 3.0], [62.51, 5.0], [63.33, 5.0], [64.19, 6.0], [64.73, 7.0], [65.84, 8.0], [66.94, 9.0], [67.67, 10.0], [69.06, 11.0], [69.91, 2.0], [70.54, 3.0], [71.55, 4.0], [72.94, 5.0], [74.12, 6.0], [75.37, 7.0], [76.66, 9.0], [77.63, 10.0], [78.7, 11.0], [79.47, 11.0], [80.31, 2.0], [81.45, 3.0], [82.47, 4.0], [83.53, 6.0], [84.44, 6.0], [85.18, 7.0], [86.48, 8.0], [87.6, 10.0], [88.32, 10.0], [89.69, 2.0], [90.52, 3.0], [91.14, 3.0], [92.12, 4.0], [92.71, 5.0], [93.08, 5.0], [93.94, 6.0], [95.06, 7.0], [95.84, 8.0], [96.35, 8.0], [97.18, 9.0], [98.06, 10.0], [99.01, 11.0], [100.14, 2.0], [101.68, 4.0], [102.64, 5.0], [104.01, 6.0], [105.23, 7.0], [106.52, 9.0], [108.1, 10.0], [108.92, 11.0], [110.0, 2.0], [110.82, 3.0], [112.12, 4.0], [113.39, 5.0], [114.36, 6.0], [115.06, 7.0], [115.89, 8.0], [117.37, 9.0], [118.11, 10.0], [118.99, 11.0], [119.94, 2.0], [120.

In [18]:
def bust_probability(dealer_cards):
    """
    dealer_cards: list -> integer series of player denoting value of cards observed upto this point

    Current body is random for now, change it accordingly
    
    output: probability of going bust on this table
    """
    bust_count = 0
    total_cards = len(dealer_cards)
    score = 0
    for card in dealer_cards:
        score += card
        if score > 16:
            if score > 21:
                bust_count += 1
            score = 0
    return bust_count/total_cards

In [21]:
bust_probability(dealer_cards)

0.07995

### Two additional points for "willingness to play"

### **1. Effective House Edge (EHE)**  
#### **Definition:**  
This metric estimates the actual house edge at a given table based on observed outcomes rather than theoretical expectations. It measures the average percentage of a player’s bet lost per round.  

#### **Formula:**  
$$
EHE = 1 - \frac{\text{Total player winnings}}{\text{Total player bets}}
$$
where:  
- **Total player winnings** = sum of all amounts won by players.  
- **Total player bets** = sum of all bets placed by players.  

#### **Strengths:**  
- Captures the real-world profitability of a table.  
- Accounts for both dealer performance and player behavior.  

#### **Weaknesses:**  
- Requires tracking actual bet sizes and payouts.  
- Can be skewed if a few players make bad decisions.  

---

### **2. Volatility Score (VS)**  
#### **Definition:**  
Measures how unpredictable the dealer’s bust rate is across different hands.  
A high volatility score suggests that the dealer’s performance is inconsistent.

#### **Formula:**  
$$
VS = \sigma_b = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (B_i - \mu_b)^2}
$$
where:  
- \( $B_i$ \) is the bust probability observed in past hands.  
- \( $\mu_b$ \) is the mean bust probability.  
- \( $\sigma_b$ \) is the standard deviation of bust probabilities.

#### **Strengths:**  
- Identifies tables with consistent dealer behavior (lower volatility is preferable).  
- Helps risk-averse players avoid unpredictable situations.

#### **Weaknesses:**  
- Doesn’t directly indicate profitability.  
- Requires enough historical data for accurate estimation.